# 6.1. [**Pipelines et estimateurs composites**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_1_compose.ipynb)</br>([*Pipelines and composite estimators*](https://scikit-learn.org/stable/modules/compose.html))

Les transformateurs sont généralement combinés avec des classifieurs, des régresseurs ou d'autres estimateurs pour construire un estimateur composite. L'outil le plus courant est un [**Pipeline** (6.1.1)](https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators). Pipeline est souvent utilisé en combinaison avec [**FeatureUnion** (6.1.3)](https://scikit-learn.org/stable/modules/compose.html#feature-union) qui concatène la sortie des transformateurs dans un espace de caractéristiques composite. [**`TransformedTargetRegressor`** (6.1.2)](https://scikit-learn.org/stable/modules/compose.html#transformed-target-regressor) traite de la transformation de la cible (c'est-à-dire log-transform y). En revanche, Pipelines ne transforme que les données observées (X).

## <a id='pipeline-chaining-estimators'></a> 6.1.1. Pipeline : chaînage d'estimateurs

[Pipeline]: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline

[**`Pipeline`**][Pipeline] peut être utilisé pour enchaîner plusieurs estimateurs en un seul. Ceci est utile car il y a souvent une séquence fixe d'étapes dans le traitement des données, par exemple la sélection des caractéristiques, la normalisation et la classification. [**`Pipeline`**][Pipeline] a plusieurs objectifs ici :

**Commodité et encapsulation**

Il suffit d'appeler `fit` et `predict` une fois sur vos données pour ajuster toute une séquence d'estimateurs.

**Sélection des paramètres communs**

Vous pouvez effectuer une [**recherche en grille** (3.2)](https://scikit-learn.org/stable/modules/grid_search.html#grid-search) sur les paramètres de tous les estimateurs du pipeline à la fois.

**Sécurité**

Les pipelines permettent d'éviter les fuites de statistiques de vos données de test dans le modèle entraîné lors de la validation croisée, en garantissant que les mêmes échantillons sont utilisés pour former les transformateurs et les prédicteurs.

Tous les estimateurs d'un pipeline, à l'exception du dernier, doivent être des transformateurs (c'est-à-dire qu'ils doivent avoir une méthode `transform`). Le dernier estimateur peut être de n'importe quel type (transformateur, classificateur, etc.).

### <a id='usage'></a> 6.1.1.1. Usage

#### <a id='construction'></a> Construction

Le [**`Pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) est construit à l'aide d'une liste de paires `(key, value)`, où `key` est une chaîne contenant le nom que vous souhaitez donner à cette étape et `value` est un objet estimateur :

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA()), ('clf', SVC())]
pipe = Pipeline(estimators)
pipe
# Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])

Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])

La fonction d'utilité [**`make_pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html) est un raccourci pour construire des pipelines; elle prend un nombre variable d'estimateurs et renvoie un pipeline, en remplissant les noms automatiquement :

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import Binarizer
make_pipeline(Binarizer(), MultinomialNB())
# Pipeline(steps=[('binarizer', Binarizer()), ('multinomialnb', MultinomialNB())])

Pipeline(steps=[('binarizer', Binarizer()), ('multinomialnb', MultinomialNB())])

#### <a id='accessing-steps'></a> Accéder aux étapes

Les estimateurs d'un pipeline sont stockés sous forme de liste dans l'attribut `steps`, mais sont accessibles par index ou nom en indexant (avec `[idx]`) le Pipeline :

In [3]:
pipe.steps[0]
# ('reduce_dim', PCA())
pipe[0]
# PCA()
pipe['reduce_dim']
# PCA()

PCA()

L'attribut `named_steps` de Pipeline permet d'accéder aux étapes par leur nom avec complétion par tabulation dans des environnements interactifs :

In [4]:
pipe.named_steps.reduce_dim is pipe['reduce_dim']
# True

True

Un sous-pipeline peut également être extrait en utilisant la notation de découpage couramment utilisée pour les séquences Python telles que les listes ou les chaînes (bien que seul un pas de 1 soit autorisé). Ceci est pratique pour n'effectuer qu'une partie des transformations (ou leur inverse) :

In [ ]:
pipe[:1]
# Pipeline(steps=[('reduce_dim', PCA())])
pipe[-1:]
# Pipeline(steps=[('clf', SVC())])

Pipeline(steps=[('clf', SVC())])

#### <a id='nested-parameters'></a> Paramètres imbriqués

Les paramètres des estimateurs du pipeline sont accessibles à l'aide de la syntaxe `<estimator>__<parameter>` :

In [5]:
pipe.set_params(clf__C=10)
# Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC(C=10))])

Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC(C=10))])

Ceci est particulièrement important pour effectuer des recherches sur grille :

In [6]:
from sklearn.model_selection import GridSearchCV
param_grid = dict(reduce_dim__n_components=[2, 5, 10],
                  clf__C=[0.1, 10, 100])
grid_search = GridSearchCV(pipe, param_grid=param_grid)

Les étapes individuelles peuvent également être remplacées en tant que paramètres, et les étapes non finales peuvent être ignorées en les définissant sur `'passthrough'` :

In [8]:
from sklearn.linear_model import LogisticRegression
param_grid = dict(
    reduce_dim=['passthrough', PCA(5), PCA(10)],
    clf=[SVC(), LogisticRegression()],
    clf__C=[0.1, 10, 100]
)
grid_search = GridSearchCV(pipe, param_grid=param_grid)

Les estimateurs du pipeline peuvent être récupérés par index :

In [9]:
pipe[0]

PCA()

ou par nom :

In [10]:
pipe['reduce_dim']

PCA()

Pour activer l'inspection du modèle, [**`Pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) a une méthode `get_feature_names_out()`, comme tous les transformateurs. Vous pouvez utiliser le découpage de pipeline pour obtenir les noms de fonctionnalités à chaque étape :

In [11]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
iris = load_iris()
pipe = Pipeline(steps=[
    ('select', SelectKBest(k=2)),
    ('clf', LogisticRegression())])
pipe.fit(iris.data, iris.target)
# Pipeline(steps=[('select', SelectKBest(...)), ('clf', LogisticRegression(...))])
pipe[:-1].get_feature_names_out()
# array(['x2', 'x3'], ...)

array(['x2', 'x3'], dtype=object)

Vous pouvez également fournir des noms de caractéristiques personnalisées pour les données d'entrée en utilisant `get_feature_names_out` :

In [12]:
pipe[:-1].get_feature_names_out(iris.feature_names)
# array(['petal length (cm)', 'petal width (cm)'], ...)

array(['petal length (cm)', 'petal width (cm)'], dtype=object)

#### Exemples

##### [**Pipeline ANOVA SVM**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/1_13_feature_selection/plot_feature_selection_pipeline.ipynb)<br/>([*Pipeline ANOVA SVM*](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_feature_selection_pipeline.html))

##### [**Exemple de pipeline pour l'extraction et l'évaluation des caractéristiques de texte**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_text_feature_extraction.ipynb)<br/>([*Sample pipeline for text feature extraction and evaluation*](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_text_feature_extraction.html))

##### [**Pipeliner: enchaîner une ACP et une régression logistique**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/6_1_compose/plot_digits_pipe.ipynb)<br/>([*Pipelining: chaining a PCA and a logistic regression*](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_digits_pipe.html))

##### [**Approximation explicite de la carte de caractéristiques pour les noyaux RBF**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/misc/plot_kernel_approximation.ipynb)<br/>([*Explicit feature map approximation for RBF kernels*](https://scikit-learn.org/stable/auto_examples/misc/plot_kernel_approximation.html))

##### [**SVM-Anova : SVM avec sélection de caractéristique univariée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/1_4_svm/plot_svm_anova.ipynb)<br/>([*SVM-Anova: SVM with univariate feature selection*](https://scikit-learn.org/stable/auto_examples/svm/plot_svm_anova.html))

##### [**Sélection de la réduction de dimensionnalité avec Pipeline et GridSearchCV**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/6_1_compose/plot_compare_reduction.ipynb)<br/>([*Selecting dimensionality reduction with Pipeline and GridSearchCV*](https://scikit-learn.org/stable/auto_examples/compose/plot_compare_reduction.html))

##### [**Affichage des pipelines**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/misc/plot_pipeline_display.ipynb)<br/>([*Displaying Pipelines*](https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_pipeline_display.html))

#### Voir également

##### [**Estimateurs composites et espaces de paramètres** (3.2.4.3)](https://scikit-learn.org/stable/modules/grid_search.html#composite-grid-search)



### <a id='notes'></a> 6.1.1.2. Notes

Appeler la méthode `fit` sur un pipeline revient à appeler la méthode `fit` sur chaque estimateur successivement, en transformant (méthode `transform`) les entrées et en la passant au prochain étape. Le pipeline possède toutes les méthodes de l'estimateur final dans le pipeline, c'est-à-dire, si l'estimateur final est un classificateur, le pipeline peut être utilisé comme un classificateur. Si l'estimateur final est un transformateur, le pipeline est également un transformateur.

### <a id='caching-transformers-avoid-repeated-computation'></a> 6.1.1.3. Mise en cache des transformateurs : éviter les calculs répétitifs

Le fitting des transformateurs peut être coûteux en termes de temps de calcul. Avec son paramètre `memory` activé, le [**`Pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) va mettre en cache chaque transformateur après avoir appelé `fit`. Cette fonctionnalité est utilisée pour éviter de calculer les transformateurs à l'intérieur d'un pipeline si les paramètres et les données d'entrée sont identiques. Un exemple typique est le cas d'une recherche de grille dans laquelle les transformateurs peuvent être ajustés une seule fois et réutilisés pour chaque configuration.

Le paramètre `memory` est nécessaire pour mettre en cache les transformateurs. `memory` peut être soit une chaîne contenant le répertoire où les transformateurs seront mis en cache, soit un objet [`joblib.Memory`](https://joblib.readthedocs.io/en/latest/memory.html).

In [13]:
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
estimators = [('reduce_dim', PCA()), ('clf', SVC())]
cachedir = mkdtemp()
pipe = Pipeline(estimators, memory=cachedir)
pipe
# Pipeline(memory=...,
#          steps=[('reduce_dim', PCA()), ('clf', SVC())])

# Clear the cache directory when you don't need it anymore
rmtree(cachedir)

**Avertissement** Effet de bord de la mise en cache des transformateurs

En utilisant un [**`Pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) sans cache activé, il est possible d'inspecter l'instance d'origine, comme :

In [14]:
from sklearn.datasets import load_digits
X_digits, y_digits = load_digits(return_X_y=True)
pca1 = PCA()
svm1 = SVC()
pipe = Pipeline([('reduce_dim', pca1), ('clf', svm1)])
pipe.fit(X_digits, y_digits)

# The pca instance can be inspected directly
print(pca1.components_)
# [[-1.77484909e-19  ... 4.07058917e-18]]

[[-1.77484909e-19 -1.73094651e-02 -2.23428835e-01 ... -8.94184677e-02
  -3.65977111e-02 -1.14684954e-02]
 [ 3.27805401e-18 -1.01064569e-02 -4.90849204e-02 ...  1.76697117e-01
   1.94547053e-02 -6.69693895e-03]
 [-1.68358559e-18  1.83420720e-02  1.26475543e-01 ...  2.32084163e-01
   1.67026563e-01  3.48043832e-02]
 ...
 [ 0.00000000e+00  3.33077242e-16  2.12297380e-16 ...  8.32667268e-17
   0.00000000e+00 -4.85722573e-17]
 [-0.00000000e+00 -2.58924789e-16 -1.40471559e-16 ... -8.32667268e-17
  -0.00000000e+00 -6.24500451e-17]
 [ 1.00000000e+00 -1.68983002e-17  5.73338351e-18 ...  8.66631300e-18
  -1.57615962e-17  4.07058917e-18]]


En activant la mise en cache, cela déclenche une copie des transformateurs avant le fitting. Par conséquent, l'instance de transformateur donnée au pipeline ne peut pas être inspectée directement. Dans l'exemple suivant, l'accès à l'instance `PCA` `pca2` déclenchera une erreur `AttributeError`, car `pca2` sera un transformateur non ajusté. Au lieu de cela, utilisez l'attribut `named_steps` pour inspecter les estimateurs à l'intérieur du pipeline :

In [15]:
cachedir = mkdtemp()
pca2 = PCA()
svm2 = SVC()
cached_pipe = Pipeline([('reduce_dim', pca2), ('clf', svm2)],
                       memory=cachedir)
cached_pipe.fit(X_digits, y_digits)
# Pipeline(memory=...,
#         steps=[('reduce_dim', PCA()), ('clf', SVC())])

print(cached_pipe.named_steps['reduce_dim'].components_)
# [[-1.77484909e-19  ... 4.07058917e-18]]

# Remove the cache directory
rmtree(cachedir)

[[-1.77484909e-19 -1.73094651e-02 -2.23428835e-01 ... -8.94184677e-02
  -3.65977111e-02 -1.14684954e-02]
 [ 3.27805401e-18 -1.01064569e-02 -4.90849204e-02 ...  1.76697117e-01
   1.94547053e-02 -6.69693895e-03]
 [-1.68358559e-18  1.83420720e-02  1.26475543e-01 ...  2.32084163e-01
   1.67026563e-01  3.48043832e-02]
 ...
 [ 0.00000000e+00  3.33077242e-16  2.12297380e-16 ...  8.32667268e-17
   0.00000000e+00 -4.85722573e-17]
 [-0.00000000e+00 -2.58924789e-16 -1.40471559e-16 ... -8.32667268e-17
  -0.00000000e+00 -6.24500451e-17]
 [ 1.00000000e+00 -1.68983002e-17  5.73338351e-18 ...  8.66631300e-18
  -1.57615962e-17  4.07058917e-18]]


#### Exemples

##### [**Sélection de la réduction de dimensionnalité avec Pipeline et GridSearchCV**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/6_1_compose/plot_compare_reduction.ipynb)<br/>([*Selecting dimensionality reduction with Pipeline and GridSearchCV*](https://scikit-learn.org/stable/auto_examples/compose/plot_compare_reduction.html))

## <a id='transforming-target-in-regression'></a> 6.1.2 Transformation de la cible en régression

[**`TransformedTargetRegressor`**](https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html) transforme les cibles `y` avant d'ajuster un modèle de régression. Les prévisions sont mappées de nouveau à l'espace d'origine via une transformation inverse. Il prend en argument le régressif qui sera utilisé pour la prédiction et le transformateur qui sera appliqué à la variable cible :

In [16]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X, y = fetch_california_housing(return_X_y=True)
X, y = X[:2000, :], y[:2000]  # select a subset of data
transformer = QuantileTransformer(output_distribution='normal')
regressor = LinearRegression()
regr = TransformedTargetRegressor(regressor=regressor,
                                  transformer=transformer)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
regr.fit(X_train, y_train)
# TransformedTargetRegressor(...)
print('R2 score: {0:.2f}'.format(regr.score(X_test, y_test)))
# R2 score: 0.61
raw_target_regr = LinearRegression().fit(X_train, y_train)
print('R2 score: {0:.2f}'.format(raw_target_regr.score(X_test, y_test)))
# R2 score: 0.59

R2 score: 0.61
R2 score: 0.59


Pour des transformations simples, au lieu d'un objet Transformer, une paire de fonctions peut être passée, définissant la transformation et son mappage inverse :

In [17]:
def func(x):
    return np.log(x)
def inverse_func(x):
    return np.exp(x)

Ensuite, l'objet est créé comme :

In [ ]:
regr = TransformedTargetRegressor(regressor=regressor,
                                  func=func,
                                  inverse_func=inverse_func)
regr.fit(X_train, y_train)
# TransformedTargetRegressor(...)
print('R2 score: {0:.2f}'.format(regr.score(X_test, y_test)))
# R2 score: 0.51

Par défaut, les fonctions fournies sont vérifiées à chaque mise en forme pour être les inverses l'une de l'autre. Cependant, il est possible de contourner cette vérification en définissant `check_inverse` sur `False`.

In [18]:
def inverse_func(x):
    return x
regr = TransformedTargetRegressor(regressor=regressor,
                                  func=func,
                                  inverse_func=inverse_func,
                                  check_inverse=False)
regr.fit(X_train, y_train)
# TransformedTargetRegressor(...)
print('R2 score: {0:.2f}'.format(regr.score(X_test, y_test)))
# R2 score: -1.57

R2 score: -1.57


**Note** : La transformation peut être déclenchée en définissant soit `transformer`, soit la paire de fonctions `func` et `inverse_func`. Cependant, définir les deux options déclenchera une erreur.

### Exemples

#### [**Effet de la transformation des cibles dans le modèle de régression**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/6_1_compose/plot_transformed_target.ipynb)<br/>([*Effect of transforming the targets in regression model*](https://scikit-learn.org/stable/auto_examples/compose/plot_transformed_target.html))

## <a id='featureunion-composite-feature-spaces'></a> 6.1.3 FeatureUnion: espaces de fonctionnalités composites

[FeatureUnion]: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html
[Pipeline]: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
[ColumnTransformer]: https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

[**`FeatureUnion`**][FeatureUnion] combine plusieurs transformateurs pour former un nouveau transformateur qui fusionne leurs sorties. Il prend en entrée une liste de transformateurs, qui sont tous ajustés aux données séparément. Les transformateurs sont appliqués en parallèle et les matrices de caractéristiques qu'ils produisent sont concaténées en une seule matrice plus grande.

Si vous souhaitez appliquer des transformations différentes à chaque champ des données, vous devriez utiliser la classe [**`ColumnTransformer`**][ColumnTransformer] correspondante (voir [**le guide de l'utilisateur** (6.1.4)](https://scikit-learn.org/stable/modules/compose.html#column-transformer)).

[**`FeatureUnion`**][FeatureUnion] a les mêmes objectifs que [**`Pipeline`**][Pipeline] : commodité, et estimation et validation des paramètres conjoints.

[**`FeatureUnion`**][FeatureUnion] et [**`Pipeline`**][Pipeline] peuvent être combinés pour créer des modèles complexes.

(Un [**`FeatureUnion`**][FeatureUnion] ne peut pas vérifier si deux transformateurs produisent des caractéristiques identiques. Il ne fusionne que des caractéristiques disjointes et c'est à l'utilisateur de s'assurer que les caractéristiques sont disjointes.

### <a id='usage'></a> 6.1.3.1. Utilisation

Un [**`FeatureUnion`**][FeatureUnion] est créé en utilisant une liste de paires `(key, value)`, où `key` est le nom que vous souhaitez donner à la transformation (une chaîne arbitraire servant uniquement d'identificateur) et `value` est un objet d'estimation. :

In [21]:
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
estimators = [('linear_pca', PCA()), ('kernel_pca', KernelPCA())]
combined = FeatureUnion(estimators)
combined
# FeatureUnion(transformer_list=[('linear_pca', PCA()),
#                                ('kernel_pca', KernelPCA())])

FeatureUnion(transformer_list=[('linear_pca', PCA()),
                               ('kernel_pca', KernelPCA())])

Comme les pipelines, les unions de caractéristiques ont un constructeur abrégé appelé [**`make_union`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_union.html) qui ne nécessite pas de nommer explicitement les composants.

Comme pour le `Pipeline`, les étapes individuelles peuvent être remplacées à l'aide de `set_params` et ignorées en les réglant sur `'drop'`:

#### Exemples

##### [**Concaténation de plusieurs méthodes d'extraction de caractéristiques**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/6_1_compose/plot_feature_union.ipynb)<br/>([*Concatenating multiple feature extraction methods*](https://scikit-learn.org/stable/auto_examples/compose/plot_feature_union.html))

## <a id='columntransformer-for-heterogeneous-data'></a> 6.1.4 ColumnTransformer pour les données hétérogènes

[Pipeline]: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
[ColumnTransformer]: https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

De nombreux ensembles de données contiennent des caractéristiques de différents types, telles que du texte, des flottants et des dates, chaque type de caractéristique nécessitant des étapes de prétraitement ou d'extraction de caractéristiques séparées. Il est souvent plus facile de prétraiter les données avant d'appliquer les méthodes scikit-learn, par exemple en utilisant [pandas](https://pandas.pydata.org/). Le traitement de vos données avant de les passer à scikit-learn peut poser problème pour l'une des raisons suivantes:
1. Incorporer des statistiques des données de test dans les prétraitements rend les scores de validation croisée peu fiables (connu sous le nom de fuite de données), par exemple dans le cas des échelles ou de l'impression de valeurs manquantes.
2. Vous souhaitez peut-être inclure les paramètres des prétraitements dans une [**recherche de paramètres** (3.2)](https://scikit-learn.org/stable/modules/grid_search.html#grid-search).

Le [**`ColumnTransformer`**][ColumnTransformer] aide à effectuer différentes transformations pour différentes colonnes des données, au sein d'un [**`Pipeline`**][Pipeline] qui est à l'abri de la fuite de données et qui peut être paramétré. Le [**`ColumnTransformer`**][ColumnTransformer] fonctionne sur les tableaux, les matrices creuses et les [pandas DataFrames](https://pandas.pydata.org/pandas-docs/stable/).

À chaque colonne, une transformation différente peut être appliquée, telle que le prétraitement ou une méthode spécifique d'extraction de caractéristiques:

In [3]:
import pandas as pd
X = pd.DataFrame(
    {'city': ['London', 'London', 'Paris', 'Sallisaw'],
     'title': ["His Last Bow", "How Watson Learned the Trick",
               "A Moveable Feast", "The Grapes of Wrath"],
     'expert_rating': [5, 3, 4, 5],
     'user_rating': [4, 5, 4, 3]})

Dans ce cas de données, nous pourrions vouloir encoder la colonne `'city'` en une variable catégorique en utilisant [**`OneHotEncoder`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) mais appliquer un [**`CountVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) à la colonne `'title'`. Comme nous pouvons utiliser plusieurs méthodes d'extraction de caractéristiques sur la même colonne, nous donnons à chaque transformateur un nom unique, disons `'city_category'` et `'title_bow'`. Par défaut, les colonnes de notation restantes sont ignorées (`remainder='drop'`):

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
column_trans = ColumnTransformer(
    [('categories', OneHotEncoder(dtype='int'), ['city']),
     ('title_bow', CountVectorizer(), 'title')],
    remainder='drop', verbose_feature_names_out=False)

column_trans.fit(X)
# ColumnTransformer(transformers=[('categories', OneHotEncoder(dtype='int'),
#                                  ['city']),
#                                 ('title_bow', CountVectorizer(), 'title')],
#                   verbose_feature_names_out=False)

column_trans.get_feature_names_out()
# array(['city_London', 'city_Paris', 'city_Sallisaw', 'bow', 'feast',
# 'grapes', 'his', 'how', 'last', 'learned', 'moveable', 'of', 'the',
#  'trick', 'watson', 'wrath'], ...)

column_trans.transform(X).toarray()
# array([[1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#        [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0],
#        [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#        [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]]...)

array([[1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]], dtype=int64)

Dans l'exemple ci-dessus, [**`CountVectorizer`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) attend une matrice 1D en entrée et donc les colonnes ont été spécifiées en tant que chaîne (`'title'`). Cependant, [**`OneHotEncoder`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html), ainsi que la plupart des autres transformateurs, attend une matrice 2D en entrée, donc dans ce cas, vous devez spécifier la colonne sous forme de liste de chaînes (`['city']`).

En dehors d'un scalaire ou d'une liste à un seul élément, la sélection de colonnes peut être spécifiée comme une liste d'éléments multiples, un tableau entier, une plage, un masque booléen ou avec un [**`make_column_selector`**](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_selector.html). Le [**`make_column_selector`**](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_selector.html) est utilisé pour sélectionner des colonnes en fonction du type de données ou du nom de colonne:

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
import numpy as np
ct = ColumnTransformer([
      ('scale', StandardScaler(),
      make_column_selector(dtype_include=np.number)),
      ('onehot',
      OneHotEncoder(),
      make_column_selector(pattern='city', dtype_include=object))])
ct.fit_transform(X)
# array([[ 0.904...,  0.      ,  1. ,  0. ,  0. ],
#        [-1.507...,  1.414...,  1. ,  0. ,  0. ],
#        [-0.301...,  0.      ,  0. ,  1. ,  0. ],
#        [ 0.904..., -1.414...,  0. ,  0. ,  1. ]])

array([[ 0.90453403,  0.        ,  1.        ,  0.        ,  0.        ],
       [-1.50755672,  1.41421356,  1.        ,  0.        ,  0.        ],
       [-0.30151134,  0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.90453403, -1.41421356,  0.        ,  0.        ,  1.        ]])

Les chaînes peuvent faire référence aux colonnes si l'entrée est un DataFrame, les entiers sont toujours interprétés comme les colonnes positionnelles.

Nous pouvons conserver les colonnes de notation restantes en définissant `remainder='passthrough'`. Les valeurs sont ajoutées à la fin de la transformation:

In [7]:
column_trans = ColumnTransformer(
    [('city_category', OneHotEncoder(dtype='int'),['city']),
     ('title_bow', CountVectorizer(), 'title')],
    remainder='passthrough')

column_trans.fit_transform(X)
# array([[1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 5, 4],
#        [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 3, 5],
#        [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 4],
#        [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 5, 3]]...)

array([[1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 5, 4],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 3, 5],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 4],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 5, 3]],
      dtype=int64)

Le paramètre `remainder` peut être défini sur un estimateur pour transformer les colonnes de notation restantes. Les valeurs transformées sont ajoutées à la fin de la transformation:

In [8]:
from sklearn.preprocessing import MinMaxScaler
column_trans = ColumnTransformer(
    [('city_category', OneHotEncoder(), ['city']),
     ('title_bow', CountVectorizer(), 'title')],
    remainder=MinMaxScaler())

column_trans.fit_transform(X)[:, -2:]
# array([[1. , 0.5],
#        [0. , 1. ],
#        [0.5, 0.5],
#        [1. , 0. ]])

array([[1. , 0.5],
       [0. , 1. ],
       [0.5, 0.5],
       [1. , 0. ]])

La fonction [**`make_column_selector`**](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_selector.html) est disponible pour créer plus facilement un objet [**`ColumnTransformer`**](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html). Plus précisément, les noms seront donnés automatiquement. L'équivalent pour l'exemple ci-dessus serait:

In [9]:
from sklearn.compose import make_column_transformer
column_trans = make_column_transformer(
    (OneHotEncoder(), ['city']),
    (CountVectorizer(), 'title'),
    remainder=MinMaxScaler())
column_trans
# ColumnTransformer(remainder=MinMaxScaler(),
#                   transformers=[('onehotencoder', OneHotEncoder(), ['city']),
#                                 ('countvectorizer', CountVectorizer(),
#                                  'title')])

ColumnTransformer(remainder=MinMaxScaler(),
                  transformers=[('onehotencoder', OneHotEncoder(), ['city']),
                                ('countvectorizer', CountVectorizer(),
                                 'title')])

Si [**`ColumnTransformer`**](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) est ajusté avec un dataframe et que le dataframe n'a que des noms de colonne en chaîne, la transformation d'un dataframe utilisera les noms de colonne pour sélectionner les colonnes:

In [10]:
ct = ColumnTransformer(
         [("scale", StandardScaler(), ["expert_rating"])]).fit(X)
X_new = pd.DataFrame({"expert_rating": [5, 6, 1],
                      "ignored_new_col": [1.2, 0.3, -0.1]})
ct.transform(X_new)
# array([[ 0.9...],
#        [ 2.1...],
#        [-3.9...]])

array([[ 0.90453403],
       [ 2.11057941],
       [-3.91964748]])

## <a id='visualizing-composite-estimators'></a> 6.1.5 Visualisation des estimateurs composites

Les estimateurs sont affichés avec une représentation HTML lorsqu'ils sont affichés dans un bloc-notes Jupyter. Cela est utile pour diagnostiquer ou visualiser un pipeline avec de nombreux estimateurs. Cette visualisation est activée par défaut:

In [11]:
column_trans  

ColumnTransformer(remainder=MinMaxScaler(),
                  transformers=[('onehotencoder', OneHotEncoder(), ['city']),
                                ('countvectorizer', CountVectorizer(),
                                 'title')])

Il peut être désactivé en définissant l'option `display` dans [**`set_config`**](https://scikit-learn.org/stable/modules/generated/sklearn.set_config.html) sur `'text'` :

In [12]:
from sklearn import set_config
set_config(display='text')  
# displays text representation in a jupyter context
column_trans  

ColumnTransformer(remainder=MinMaxScaler(),
                  transformers=[('onehotencoder', OneHotEncoder(), ['city']),
                                ('countvectorizer', CountVectorizer(),
                                 'title')])

Un exemple de la sortie HTML peut être vu dans la **représentation HTML de la section Pipeline** de [**Column Transformer avec Mixed Types**](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html). En alternative, l'HTML peut être écrit dans un fichier à l'aide de [**`estimator_html_repr`**](https://scikit-learn.org/stable/modules/generated/sklearn.utils.estimator_html_repr.html).

In [13]:
from sklearn.utils import estimator_html_repr
with open('my_estimator.html', 'w') as f:  
    f.write(estimator_html_repr(clf))

NameError: name 'clf' is not defined

### Exemples

#### [**Transformateur de colonne avec des sources de données hétérogènes**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/6_1_compose/plot_column_transformer.ipynb)<br/>([*Column Transformer with Heterogeneous Data Sources*](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer.html))

#### [**Transformation de colonne avec des types mélangés**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/6_1_compose/plot_column_transformer_mixed_types.ipynb)<br/>([*Column Transformer with Mixed Types*](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html))